In [1]:
"""
Lesson 6: Tool Integration - Interactive Notebook
==================================================
Save as: notebooks/lesson_06_tools_experiments.py
or convert to .ipynb for Jupyter

This notebook walks through building and using tools with hands-on examples.
"""

import json
import time
from typing import Any, Dict, List, Optional
from dataclasses import dataclass
import random

In [2]:
# ============================================================================
# PART 1: Understanding Tools - The Basics
# ============================================================================

def example_1_tool_basics():
    """
    Example: What is a tool?
    """
    print("=" * 60)
    print("EXAMPLE 1: Tool Basics")
    print("=" * 60)
    
    # Without tools - just text
    print("\n🤖 Agent WITHOUT tools:")
    agent_response = "I will search for 'AI agents' and analyze the results"
    print(f"Agent says: {agent_response}")
    print("Result: Just words, no action! ❌")
    
    # With tools - actual action
    print("\n🤖 Agent WITH tools:")
    
    # Simple tool function
    def search_tool(query: str) -> list:
        """Simulate web search"""
        # In real life, this would call a search API
        mock_results = [
            {"title": "What are AI Agents?", "url": "example.com/1"},
            {"title": "Building AI Agents", "url": "example.com/2"},
            {"title": "Agent Frameworks", "url": "example.com/3"}
        ]
        print(f"🔍 Searching for: {query}")
        return mock_results
    
    # Agent actually uses the tool
    results = search_tool("AI agents")
    print(f"✅ Found {len(results)} results:")
    for r in results:
        print(f"  - {r['title']}")
    
    print("\n💡 Key Insight: Tools = Functions that DO things!")
    print("-" * 60)

In [ ]:
# ============================================================================
# PART 2: Building a Simple Tool
# ============================================================================

@dataclass
class ToolResult:
    """Standardized tool output"""
    success: bool
    data: Any = None
    error: Optional[str] = None

class SimpleTool:
    """A basic tool structure"""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
    
    def execute(self, **kwargs) -> ToolResult:
        """Override this in subclasses"""
        raise NotImplementedError

def example_2_simple_tool():
    """
    Example: Building a simple calculator tool
    """
    print("\n" + "=" * 60)
    print("EXAMPLE 2: Building a Simple Tool")
    print("=" * 60)
    
    class CalculatorTool(SimpleTool):
        """A calculator that agents can use"""
        
        def __init__(self):
            super().__init__(
                name="calculator",
                description="Performs basic math operations: add, subtract, multiply, divide"
            )
        
        def execute(self, operation: str, a: float, b: float) -> ToolResult:
            """Execute the calculation"""
            try:
                if operation == "add":
                    result = a + b
                elif operation == "subtract":
                    result = a - b
                elif operation == "multiply":
                    result = a * b
                elif operation == "divide":                    if b == 0:
                        return ToolResult(
                            success=False,
                            error="Cannot divide by zero"
                        )
                    result = a / b
                else:
                    return ToolResult(
                        success=False,
                        error=f"Unknown operation: {operation}"
                    )
                
                return ToolResult(success=True, data=result)
                
            except Exception as e:
                return ToolResult(success=False, error=str(e))
    
    # Use the tool
    calc = CalculatorTool()
    
    print(f"\n🛠️  Tool: {calc.name}")
    print(f"📝 Description: {calc.description}")
    
    # Test calculations
    tests = [
        ("add", 10, 5),
        ("multiply", 7, 8),
        ("divide", 100, 4),
        ("divide", 10, 0),  # This will fail
    ]
    
    print("\n🧪 Testing the tool:")
    for op, a, b in tests:
        result = calc.execute(operation=op, a=a, b=b)
        
        if result.success:
            print(f"✅ {a} {op} {b} = {result.data}")
        else:
            print(f"❌ {a} {op} {b} failed: {result.error}")
    
    print("\n💡 Key Insight: Tools have clear inputs and outputs")
    print("-" * 60)

In [4]:
# ============================================================================
# PART 3: Error Handling in Tools
# ============================================================================

def example_3_error_handling():
    """
    Example: Why error handling matters
    """
    print("\n" + "=" * 60)
    print("EXAMPLE 3: Error Handling")
    print("=" * 60)
    
    class WebSearchTool(SimpleTool):
        """Search tool with error handling"""
        
        def __init__(self):
            super().__init__(
                name="web_search",
                description="Search the web for information"
            )
            self.call_count = 0
        
        def execute(self, query: str) -> ToolResult:
            """Search with various error scenarios"""
            self.call_count += 1
            
            # Simulate different error scenarios
            if not query or query.strip() == "":
                return ToolResult(
                    success=False,
                    error="Query cannot be empty"
                )
            
            # Simulate rate limiting (every 5th call fails)
            if self.call_count % 5 == 0:
                return ToolResult(
                    success=False,
                    error="Rate limit exceeded. Try again in 60 seconds."
                )
            
            # Simulate network error (10% chance)
            if random.random() < 0.1:
                return ToolResult(
                    success=False,
                    error="Network error. Please check your connection."
                )
            
            # Success case
            mock_results = [
                {"title": f"Result for '{query}'", "url": "example.com"}
            ]
            
            return ToolResult(success=True, data=mock_results)
    
    # Test error handling
    search = WebSearchTool()
    
    print("\n🧪 Testing various scenarios:\n")
    
    # Test 1: Empty query
    result = search.execute("")
    print(f"Test 1 - Empty query: {'✅ Handled' if not result.success else '❌ Should fail'}")
    print(f"  Error: {result.error}\n")
    
    # Test 2: Multiple calls (to hit rate limit)
    print("Test 2 - Rate limiting:")
    for i in range(6):
        result = search.execute(f"query {i}")
        if result.success:
            print(f"  Call {i+1}: ✅ Success")
        else:
            print(f"  Call {i+1}: ⚠️  {result.error}")
    
    print("\n💡 Key Insight: Always expect tools to fail!")
    print("-" * 60)

In [5]:
# ============================================================================
# PART 4: Tool Types and Use Cases
# ============================================================================

def example_4_tool_types():
    """
    Example: Different types of tools
    """
    print("\n" + "=" * 60)
    print("EXAMPLE 4: Different Tool Types")
    print("=" * 60)
    
    # 1. Information Retrieval Tool
    class WeatherTool(SimpleTool):
        def execute(self, location: str) -> ToolResult:
            weather_data = {
                "location": location,
                "temperature": 72,
                "condition": "Sunny",
                "humidity": 65
            }
            return ToolResult(success=True, data=weather_data)
    
    # 2. Action/Mutation Tool
    class FileWriteTool(SimpleTool):
        def execute(self, filename: str, content: str) -> ToolResult:
            # Simulate writing to file
            print(f"  📝 Writing to {filename}...")
            return ToolResult(
                success=True,
                data=f"Wrote {len(content)} characters to {filename}"
            )
    
    # 3. Computation Tool
    class DataAnalyzerTool(SimpleTool):
        def execute(self, data: List[int]) -> ToolResult:
            analysis = {
                "count": len(data),
                "sum": sum(data),
                "average": sum(data) / len(data) if data else 0,
                "max": max(data) if data else None,
                "min": min(data) if data else None
            }
            return ToolResult(success=True, data=analysis)
    
    # Test each type
    print("\n🔍 Type 1: Information Retrieval")
    weather = WeatherTool("weather", "Get current weather")
    result = weather.execute("New York")
    print(f"  {json.dumps(result.data, indent=2)}")
    
    print("\n✍️  Type 2: Action/Mutation")
    file_writer = FileWriteTool("file_writer", "Write to files")
    result = file_writer.execute("report.txt", "Agent analysis results...")
    print(f"  {result.data}")
    
    print("\n🧮 Type 3: Computation")
    analyzer = DataAnalyzerTool("analyzer", "Analyze data")
    result = analyzer.execute([10, 20, 30, 40, 50])
    print(f"  {json.dumps(result.data, indent=2)}")
    
    print("\n💡 Key Insight: Different tools for different needs!")
    print("-" * 60)

In [6]:
# ============================================================================
# PART 5: Tool Composition - Combining Tools
# ============================================================================

def example_5_tool_composition():
    """
    Example: Using multiple tools together
    """
    print("\n" + "=" * 60)
    print("EXAMPLE 5: Tool Composition")
    print("=" * 60)
    
    # Simple tools
    class SearchTool(SimpleTool):
        def execute(self, query: str):
            results = [
                {"title": "Article 1", "url": "example.com/1"},
                {"title": "Article 2", "url": "example.com/2"}
            ]
            return ToolResult(success=True, data=results)
    
    class FetchTool(SimpleTool):
        def execute(self, url: str):
            content = f"Content from {url}..."
            return ToolResult(success=True, data=content)
    
    class SummarizeTool(SimpleTool):
        def execute(self, text: str):
            summary = f"Summary of: {text[:30]}..."
            return ToolResult(success=True, data=summary)
    
    # Compose them into a workflow
    def research_workflow(topic: str):
        """Multi-step research using tool composition"""
        print(f"\n📚 Researching: {topic}\n")
        
        # Step 1: Search
        print("Step 1: 🔍 Searching...")
        search = SearchTool("search", "Search web")
        search_result = search.execute(topic)
        
        if not search_result.success:
            return "Search failed"
        
        print(f"  Found {len(search_result.data)} results")
        
        # Step 2: Fetch top results
        print("\nStep 2: 📥 Fetching content...")
        fetch = FetchTool("fetch", "Fetch URLs")
        contents = []
        
        for result in search_result.data[:2]:
            fetch_result = fetch.execute(result['url'])
            if fetch_result.success:
                contents.append(fetch_result.data)
                print(f"  ✅ Fetched: {result['title']}")
        
        # Step 3: Summarize
        print("\nStep 3: 📊 Summarizing...")
        summarize = SummarizeTool("summarize", "Summarize text")
        
        all_content = " ".join(contents)
        summary_result = summarize.execute(all_content)
        
        if summary_result.success:
            print(f"  ✅ {summary_result.data}")
        
        return summary_result.data
    
    # Run the workflow
    result = research_workflow("AI Agents")
    
    print("\n💡 Key Insight: Combine simple tools for complex tasks!")
    print("-" * 60)


In [7]:
# ============================================================================
# PART 6: Building a Tool Registry
# ============================================================================

class SimpleToolRegistry:
    """Manage multiple tools"""
    
    def __init__(self):
        self.tools: Dict[str, SimpleTool] = {}
    
    def register(self, tool: SimpleTool):
        """Add a tool to the registry"""
        self.tools[tool.name] = tool
        print(f"✅ Registered: {tool.name}")
    
    def get(self, name: str) -> Optional[SimpleTool]:
        """Get a tool by name"""
        return self.tools.get(name)
    
    def list_tools(self) -> List[str]:
        """List all available tools"""
        return list(self.tools.keys())
    
    def describe_tools(self) -> str:
        """Get descriptions of all tools"""
        descriptions = []
        for tool in self.tools.values():
            descriptions.append(f"- {tool.name}: {tool.description}")
        return "\n".join(descriptions)

def example_6_tool_registry():
    """
    Example: Managing multiple tools
    """
    print("\n" + "=" * 60)
    print("EXAMPLE 6: Tool Registry")
    print("=" * 60)
    
    # Create registry
    registry = SimpleToolRegistry()
    
    # Create various tools
    class SearchTool(SimpleTool):
        def __init__(self):
            super().__init__("search", "Search the web")
    
    class CalculatorTool(SimpleTool):
        def __init__(self):
            super().__init__("calculator", "Perform calculations")
    
    class WeatherTool(SimpleTool):
        def __init__(self):
            super().__init__("weather", "Get weather information")
    
    # Register tools
    print("\n📦 Registering tools:")
    registry.register(SearchTool())
    registry.register(CalculatorTool())
    registry.register(WeatherTool())
    
    # List available tools
    print(f"\n🛠️  Available tools: {registry.list_tools()}")
    
    # Describe tools
    print("\n📋 Tool descriptions:")
    print(registry.describe_tools())
    
    # Get specific tool
    print("\n🔍 Getting 'search' tool:")
    search = registry.get("search")
    if search:
        print(f"  Found: {search.name} - {search.description}")
    
    print("\n💡 Key Insight: Registry makes tools discoverable!")
    print("-" * 60)

In [8]:
# ============================================================================
# PART 7: Agent Using Tools
# ============================================================================

class SimpleAgent:
    """Agent that can use tools"""
    
    def __init__(self, name: str, tools: SimpleToolRegistry):
        self.name = name
        self.tools = tools
    
    def execute_task(self, task: str):
        """Simulate agent deciding which tool to use"""
        print(f"\n🤖 {self.name} received task: '{task}'")
        
        # Simple keyword matching to choose tool
        if "search" in task.lower():
            tool = self.tools.get("search")
            print(f"  💭 Thinking: I need to search for information")
            print(f"  🛠️  Using tool: {tool.name}")
            
        elif "calculate" in task.lower() or "math" in task.lower():
            tool = self.tools.get("calculator")
            print(f"  💭 Thinking: This requires calculation")
            print(f"  🛠️  Using tool: {tool.name}")
            
        elif "weather" in task.lower():
            tool = self.tools.get("weather")
            print(f"  💭 Thinking: User wants weather info")
            print(f"  🛠️  Using tool: {tool.name}")
        else:
            print(f"  ❌ I don't have the right tool for this task")
            return None
        
        return tool

def example_7_agent_with_tools():
    """
    Example: Agent using tools to accomplish tasks
    """
    print("\n" + "=" * 60)
    print("EXAMPLE 7: Agent Using Tools")
    print("=" * 60)
    
    # Set up tools
    registry = SimpleToolRegistry()
    
    class SearchTool(SimpleTool):
        def __init__(self):
            super().__init__("search", "Search the web")
    
    class CalculatorTool(SimpleTool):
        def __init__(self):
            super().__init__("calculator", "Perform calculations")
    
    class WeatherTool(SimpleTool):
        def __init__(self):
            super().__init__("weather", "Get weather information")
    
    registry.register(SearchTool())
    registry.register(CalculatorTool())
    registry.register(WeatherTool())
    
    # Create agent
    agent = SimpleAgent("ResearchBot", registry)
    
    # Give agent various tasks
    tasks = [
        "Search for AI research papers",
        "Calculate the sum of 45 and 67",
        "What's the weather in Tokyo?",
        "Write a poem"  # Agent doesn't have this tool
    ]
    
    for task in tasks:
        tool = agent.execute_task(task)
        if tool:
            print(f"  ✅ Task can be accomplished")
        else:
            print(f"  ⚠️  Cannot complete this task")
        print()
    
    print("💡 Key Insight: Agents choose tools based on the task!")
    print("-" * 60)


In [9]:
# ============================================================================
# PART 8: Exercises - Your Turn!
# ============================================================================

def exercises():
    """
    Practice exercises for you to try
    """
    print("\n" + "=" * 60)
    print("🎯 YOUR TURN - EXERCISES")
    print("=" * 60)
    
    exercises = [
        {
            "name": "Exercise 1: Build a Custom Tool",
            "task": "Create a 'TranslatorTool' that translates text between languages",
            "hint": "Use a dictionary for simple translations, or mock an API call"
        },
        {
            "name": "Exercise 2: Add Safety Checks",
            "task": "Add input validation to prevent invalid parameters",
            "hint": "Check for empty strings, negative numbers where not allowed, etc."
        },
        {
            "name": "Exercise 3: Rate Limiting",
            "task": "Implement a rate limiter for a tool (max 5 calls per minute)",
            "hint": "Track timestamps of calls and check before executing"
        },
        {
            "name": "Exercise 4: Tool Chaining",
            "task": "Create a workflow that uses 3+ tools in sequence",
            "hint": "Output of one tool becomes input to the next"
        },
        {
            "name": "Exercise 5: Smart Tool Selection",
            "task": "Improve the agent's tool selection logic using better parsing",
            "hint": "Look for action verbs, entities, and context in the task"
        }
    ]
    
    for i, ex in enumerate(exercises, 1):
        print(f"\n{ex['name']}")
        print(f"  📝 {ex['task']}")
        print(f"  💡 Hint: {ex['hint']}")
    
    print("\n" + "=" * 60)
    print("\n🎉 Try these exercises to master tool building!")

# ============================================================================
# MAIN: Run All Examples
# ============================================================================

if __name__ == "__main__":
    print("\n")
    print("*" * 60)
    print("  LESSON 6: TOOL INTEGRATION - INTERACTIVE NOTEBOOK")
    print("*" * 60)
    
    # Run all examples
    example_1_tool_basics()
    example_2_simple_tool()
    example_3_error_handling()
    example_4_tool_types()
    example_5_tool_composition()
    example_6_tool_registry()
    example_7_agent_with_tools()
    exercises()
    
    print("\n\n🎉 Notebook complete! Now try the exercises and build your own tools.\n")
    print("💡 Next: Check out the production tool system in prototypes/tool_system.py\n")



************************************************************
  LESSON 6: TOOL INTEGRATION - INTERACTIVE NOTEBOOK
************************************************************
EXAMPLE 1: Tool Basics

🤖 Agent WITHOUT tools:
Agent says: I will search for 'AI agents' and analyze the results
Result: Just words, no action! ❌

🤖 Agent WITH tools:
🔍 Searching for: AI agents
✅ Found 3 results:
  - What are AI Agents?
  - Building AI Agents
  - Agent Frameworks

💡 Key Insight: Tools = Functions that DO things!
------------------------------------------------------------

EXAMPLE 2: Building a Simple Tool

🛠️  Tool: calculator
📝 Description: Performs basic math operations: add, subtract, multiply, divide

🧪 Testing the tool:
✅ 10 add 5 = 15
✅ 7 multiply 8 = 56
✅ 100 divide 4 = 25.0
❌ 10 divide 0 failed: Cannot divide by zero

💡 Key Insight: Tools have clear inputs and outputs
------------------------------------------------------------

EXAMPLE 3: Error Handling

🧪 Testing various scenarios:

Te